# Bitcoin Price Prediction
using time series analysis

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import re

In [2]:
df = pd.read_csv('./data/bitcoin_eda.csv', index_col=0)
df.head()

,date,close,volume,fluctuation,weighted_close,close_log,close_norm,volume_norm,weighted_close_norm
0,2013-01-01,13.3,28410.0,-1.55,377853.0,2.587764,0.000000,0.028034,0.000000e+00
1,2013-01-02,13.3,17970.0,0.00,239001.0,2.587764,0.000000,0.017585,0.000000e+00
2,2013-01-03,13.4,18030.0,0.90,241602.0,2.595255,0.000002,0.017645,3.073006e-08
3,2013-01-04,13.5,29640.0,0.75,400140.0,2.602690,0.000003,0.029265,1.019338e-07
4,2013-01-05,13.4,21330.0,-0.44,285822.0,2.595255,0.000002,0.020948,3.648214e-08


## 정상 시계열
정상적시계열(Stationary Time Series)
- 실제 시계열은 추세,계절성을포함하는 비정상적(non-stationary)이 대다수
- 우선 정상적 시계열의 성질을 알아본다.
- 비정상적 시계열은 적절한 변환을 통하여 정상적 시계열로 바꿀 수 있다.

### 강정상성(Strong Stationarity)
- 시계열 $\{Z_t, t \geq 1\}$에대해 $(Z_1, ..., Z_m)$과 $(Z_{1+k},...,Z_{m+k})$ 이 동일한 결합확률분포를 가질때 강정상성을 갖는 시계열이라 함
- 기대치가 시간에 따라 일정
- 분산이 시간에 따라 일정
- 자기공분산 또는 자기상관계수가 시간간격(time lag)에만 의존

### 약정상성(Weak Stationarity):
- 시계열 $\{Z_t, t \geq 1\}$에 대해 기대치가 시간에 따라 일정하고 임의 두 시점 자기공분산이 시간 간격에만 의존하고 유한할때 약정상성을 갖는 시계열이라 함
- 강정상성이 성립하면 약정상성 성립; 역은 성립하지 않음
- 결합확률분포가 다변량정규분포를 따를 때 강정상성과 약정상성은 일치
- 시계열 분석에서는 주로 약정상성을 가정함

### 자기공분산 (autocovariance)
- 시계열의 시간에 따른 연관패턴을 자기공분산으로 요약
- 시차 k의 자기공분산 (**약 정상성 가정**)

$$\gamma (k) = Cov[Z_t, Z_{t-k}] = E[(Z_t-\mu _z)(Z_{t-k}-\mu _z)] \\
\gamma (0) = Var[Z_t] \\
\gamma (k) = \gamma (-k)
$$

$\gamma (k)$를 k의 함수(k=0,1,2,...)로 볼 때, $\gamma (k)$를 자기공분산 함수(autocovariance function)이라 함.

### 자기상관함수 (Autocorrelation Function; ACF)
시차 k의 자기상관계수:
$$
\rho (k) = Corr[Z_t, Z_{t-k}]=\frac{Cov[Z_t, Z_{t-k}]}{Var[Z_t]}=\frac{\gamma (k)}{\gamma (0)} \\ 
\rho (0) = 1 \\
\rho (k) = \rho (-k)
$$

$\rho (k)$를 k의 함수(k=0,1,2,...)로 볼 때, 자기상관함수(ACF)라고 부름
- 비교적 단순한 형태의 정상 시계열 모형을 주로 다루며, **ACF로 모형을 식별함**

### 편자기상관함수 (Partial Autocorrelation Function, PACF)
- 시계열 모형의 정상성 식별에 ACF 외에 PACF 정보를 활용함
- PACF란 시차가 k인 두 값들 간의 상관계수가 중간 시점들의 값들이 이미 설명된 이후 추가적 영향만을 고려하기 위하여 고안된 것
- 따라서 다음과 같은 조건부 상관계수를 고려함

$$P(k) = Corr[Z_t, Z_{t-k}|Z_{t-1}, ..., Z_{t-k+1}] \\ for \ k=0, 1, 2, ... $$
- $P(1) = \rho (1)$, 중간 값이 없으므로
- $P(k)$는 다음 회귀모형의 계수 $\phi_{kk}$와 동일함

$$Z_t = \phi_{k1}Z_{t-1}+\phi_{k2}Z_{t-2}+...+\phi_{kk}Z_{t-k}+b_t$$

## 시계열의 표현방식 : AR, MA

### 자기회귀(autoregressive; AR)

$$Z_t = \phi_1 Z_{t-1}+\phi_2 Z_{t-2} + ... + a_t$$

- 시점 t의 값($Z_t$)을 과거시점의 값들을 이용한 **회귀식**으로 표현
- $\phi_1, \phi_2$는 상수이며, $a_t$는 백색잡음 (white noise)
    - AR(1) = 시차 1 변수 포함 = $\phi_1 Z_{t-1}+a_t$

<br>

#### AR 모형과 정상성 조건
- AR(1)
    - $Z_t = \phi_1 Z_{t-1}+a_t$
        - $a_t$ = 독립 오차항 = 평균 0, 분산 $\sigma ^2$의 백색잡음
        - 정상성을 갖기 위해 모수 $\phi_1$에 대한 조건 필요
            - $-1 < \phi_1 < 1$
- AR(2)
    - $Z_t = \phi_1 Z_{t-1}+\phi_2 Z_{t-2}+a_t$
    - 정상성 조건 (이하 동시에 충족)
        - $-1 < \phi_2 < 1$
        - $\phi_1+\phi_2<1$
        - $\phi_2-\phi_1<1$
- AR(p)
    - $Z_t = \phi_1 Z_{t-1}+\phi_2 Z_{t-2}+...+\phi_p Z_{t-p}+a_t$
    - 정상성 조건
        - 다항식 $\Phi_p(x)=0$이 갖는 p개의 근의 크기(modulus)가 모두 1보다 커야 함

<br>

#### AR 모형의 ACF / PACF
AR(p=1) 기준
- ACF
    - 지수적으로 감소하는 패턴
- PACF
    - 시차 p(=1) 이후 절단 패턴

![](./img/ar1-acf-pacf.png)

### 이동평균(moving average; MA)

$$Z_t = a_t - \theta_1 a_{t-1}-\theta_2 a_{t-2}-...$$

- 시점 t의 값($Z_t$)을 현재와 과거시점의 백색잡음으로 표현
- $\theta_1, \theta_2$ 등은 상수
    - MA(1) = 시차 1 백색잡음 포함 = $Z_t = a_t - \theta_1 a_{t-1}$
        
<br>
        
#### MA 모형과 가역성 조건 
MA 모형은 정상성을 가지며, AR형태로 표현하기 위해 모수 $\theta_1$에 대한 가역성(invertibility) 조건 필요

- MA(1)
    - 시점 t의 값은 시점 t와 시점 t-1의 오차항으로 생성
    - $Z_t = a_t - \theta_1 a_{t-1}$
    - $a_t$ = 독립 오차항 = 평균 0, 분산 $\sigma ^2$의 백색잡음
    - 가역성 조건
        - $-1 < \theta_1 < 1$
- MA(2)
    - 시차 2 백색잡음까지 포함
    - $Z_t = a_t - \theta_1 a_{t-1} - \theta_2 a_{t-2}$
    - 가역성 조건
        - $-1 < \theta_2 < 1$
        - $\theta_1+\theta_2<1$
        - $\theta_2-\theta_1<1$
- MA(q)
    - 시차 q 백색잡음까지 포함
    - $Z_t = a_t - \theta_1 a_{t-1} - \theta_2 a_{t-2} - ... - \theta_q a_{t-q}$
    - 가역성 조건
        - 다항식 $\Theta_q(x)=0$이 갖는 q개의 근의 크기(modulus)가 모두 1보다 커야 함

<br>

#### MA 모형의 ACF / PACF
MA(q=1) 기준
- ACF
    - 시차 q(=1) 이후 절단 패턴
- PACF
    - 지수적으로 감소하는 패턴

![](./img/ma1-acf-pacf.png)

### ARMA 모형
- AR 방식과 MA 방식을 결합한 모형

<br>

#### ARMA 모형과 정상성/가역성 조건
- ARMA(1, 1) : AR(1) + MA(1)
    - $Z_t - \phi_1 Z_{t-1} = a_t- \theta a_{t-1}$
    - 시점 t의 값은 시점 t-1의 값, 그리고 시점 t와 t-1의 오차항으로 생성됨
    - 정상성 조건
        - $-1 < \phi_1 < 1$
    - 가역성 조건
        - $-1 < \theta_1 < 1$
- ARMA(p, q) : 시차 p까지의 변수와 시차 q까지의 오차항을 포함

- $$Z_t - \phi_1 Z_{t-1} - ... - \phi_p Z_{t-p} = a_t- \theta a_{t-1} - .... - \theta_q a_{t-p} \\ \Rightarrow \Phi_p (B)Z_t=\Theta_q (B)a_t$$

    - 정상성 조건
        - 다항식 $\Phi_p(x)=0$이 갖는 p개의 근의 크기(modulus)가 모두 1보다 커야 함
    - 가역성 조건
        - 다항식 $\Theta_q(x)=0$이 갖는 q개의 근의 크기(modulus)가 모두 1보다 커야 함
        
<br>

#### ARMA 모형의 ACF / PACF

ACF

$$\rho (k) = \phi_1 \rho(k-1)+...+\phi_p \rho (k-p), \\ k \geq max(p, q+1)$$

- $p \geq q+1$ 일 때, ACF는 AR(q) 모형과 유사하게 0으로 떨어진다.
- $p \leq q$ 일 때, 처음 $q-p$ 값은 별개의 값을 갖고 이후 AR(p) 모형과 유사하게 0으로 떨어진다.

<br>

PACF
- $p \geq q+1$ 일 때, PACF는 처음 $p-q$값은 별개의 값을 갖고 이후 MA(q) 모형과 유사하게 0으로 떨어진다.
- $p \leq q$ 일 때, 처음부터 MA(q) 모형과 유사하게 0으로 떨어진다.

<br>


ACF/PACF - ARIMA(1,3)
- $q-p=2$
    - ACF 처음 2개는 별개의 값, 이후 지수적으로 감소
    - PACF는 처음부터 지수적으로 감소
![](./img/arima13-acf-pacf.png)

ACF/PACF - ARIMA(3,1)
- $p-q=2$
    - ACF는 처음부터 지수적으로 감소
    - PACF는 처음 2개 별개의 값, 이후 지수적으로 감소
![](./img/arima31-acf-pacf.png)

### ARMA 모형의 이론적 ACF / PACF 패턴


모형|ACF|PACF
-|-|-
AR(p)|지수적으로 감소|시차 p 이후 절단
MA(q)|시차 q 이후 절단|지수적으로 감소
ARMA(p,q)|시차 (p-q) 이후 지수적 감소|시차(p-q) 이후 지수적 감소